In [1]:
import numpy as np
import pandas as pd

import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col

def getSparkSession():
    return SparkSession.builder.config('spark.local.dir', '/tmp').config("spark.executor.memory", "8g").config("spark.driver.memory", "8g").config("spark.driver.maxResultSize", "0").config("spark.shuffle.registration.maxAttempts", "1").config("spark.task.maxFailures", "1").appName("jupyter").getOrCreate()
    #return SparkSession.builder.config('spark.local.dir', '/tmp').config("spark.executor.memory", "8g").config("spark.driver.memory", "8g").config("spark.driver.maxResultSize", "0").appName("jupyter").master("spark://clspromon-aio01.txx.seeburger.de:7077").getOrCreate()
    #return SparkSession.builder.config('spark.local.dir', '/tmp').config("spark.executor.memory", "8g").config("spark.driver.memory", "8g").config("spark.driver.maxResultSize", "0").appName("example-pyspark-read-and-write").appName('jupyter').master('spark://clspromon-aio01.txx.seeburger.de:7077').getOrCreate()
    #return SparkSession.builder.config('spark.local.dir', '/tmp').config("spark.executor.memory", "8g").config("spark.driver.memory", "8g").config("spark.driver.maxResultSize", "0").appName("example-pyspark-read-and-write").appName('jupyter').master('spark://172.30.17.145:7077').getOrCreate()


In [2]:
import os
os.environ["HADOOP_USER_NAME"] = "hdfs"

In [3]:
start_year=2022
start_month=7
start_day_in_month=12

end_year=2022
end_month=7
end_day_in_month=15

In [4]:
columns = ['CGLOBALMESSAGEID', 'CSTARTTIME', 'CENDTIME', 'CSTATUS', 'CSERVICE',\
       'CSLABILLINGMONTH', 'CSENDERPROTOCOL', 'CSENDERENDPOINTID',\
       'CINBOUNDSIZE', 'CRECEIVERPROTOCOL', 'CRECEIVERENDPOINTID', 'CSLATAT',\
       'CMESSAGETAT2', 'CSLADELIVERYTIME']
# withot 'CSLABILLINGMONTH'
def get_columns_2():
    columns = ['CGLOBALMESSAGEID', 'CSTARTTIME', 'CENDTIME', 'CSTATUS', 'CSERVICE',\
            'CSENDERPROTOCOL', 'CSENDERENDPOINTID',\
           'CINBOUNDSIZE', 'CRECEIVERPROTOCOL', 'CRECEIVERENDPOINTID', 'CSLATAT',\
           'CMESSAGETAT2', 'CSLADELIVERYTIME']
    return columns

columns = ['CGLOBALMESSAGEID',  'CSTARTTIME', 'CENDTIME', 'CSTATUS', 'CSERVICE', 'CSENDERENDPOINTID', 'CSENDERPROTOCOL', 'CINBOUNDSIZE', 'CRECEIVERPROTOCOL', 'CRECEIVERENDPOINTID', 'CSLATAT', 'CMESSAGETAT2', 'CSLADELIVERYTIME']
     


In [5]:
sparkSession = getSparkSession()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/01 16:51:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/01 16:51:26 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [6]:
from datetime import datetime, timezone
import pyspark.sql.functions as f

def timestamp(dt):
    return dt.replace(tzinfo=timezone.utc).timestamp() * 1000

startTime = timestamp(datetime(start_year, start_month, start_day_in_month))
endTime = timestamp(datetime(end_year, end_month, end_day_in_month))

print(endTime)

filename_min=str(int(startTime/10000000))

print(filename_min)

1657843200000.0
165758


In [7]:
import pyspark.sql.functions as f
df = sparkSession.read.parquet("hdfs://172.30.17.145:8020/sla_sql_data/" + filename_min +"*/*").select(columns).filter((f.col('CSTARTTIME') >= startTime) & (f.col('CSTARTTIME') < endTime)).sort(f.col('CSTARTTIME')).dropDuplicates()

In [8]:
df.printSchema()

root
 |-- CGLOBALMESSAGEID: string (nullable = true)
 |-- CSTARTTIME: decimal(38,0) (nullable = true)
 |-- CENDTIME: decimal(38,0) (nullable = true)
 |-- CSTATUS: string (nullable = true)
 |-- CSERVICE: string (nullable = true)
 |-- CSENDERENDPOINTID: string (nullable = true)
 |-- CSENDERPROTOCOL: string (nullable = true)
 |-- CINBOUNDSIZE: decimal(38,0) (nullable = true)
 |-- CRECEIVERPROTOCOL: string (nullable = true)
 |-- CRECEIVERENDPOINTID: string (nullable = true)
 |-- CSLATAT: decimal(38,0) (nullable = true)
 |-- CMESSAGETAT2: decimal(38,0) (nullable = true)
 |-- CSLADELIVERYTIME: decimal(38,0) (nullable = true)



In [9]:
import pandas as pd
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
df_pandas= df.toPandas()

df_pandas

,CGLOBALMESSAGEID,CSTARTTIME,CENDTIME,CSTATUS,CSERVICE,CSENDERENDPOINTID,CSENDERPROTOCOL,CINBOUNDSIZE,CRECEIVERPROTOCOL,CRECEIVERENDPOINTID,CSLATAT,CMESSAGETAT2,CSLADELIVERYTIME
0,d02228b0-017e-11ed-84c3-49e4ac1e100f,1657587969449,1657587973838,SUCCESS,MAKO_BE_ROUTER,e57c94c0-e0f0-11e8-be62-528eac1b495c,AS2,198,AS2,9eeb0b20-a253-11e9-a189-ddafac1b495c,4302,4389,1657587973751
1,4df52828-017f-11ed-84c3-49e4ac1e100f,1657588180772,1657588186259,SUCCESS,MAKO_BE_ROUTER,e57c94c0-e0f0-11e8-be62-528eac1b495c,AS2,185,AS2,88d4b600-3019-11ec-8a09-23efac1e124c,5456,5487,1657588186228
2,95cfeea3-017f-11ed-9366-c1f5ac1e100e,1657588301089,1657588304140,SUCCESS,MAKO_BE_ROUTER,e57c94c0-e0f0-11e8-be62-528eac1b495c,AS2,65575,REST,f9641661-3874-11eb-98c1-b429ac1b495c,3006,3051,1657588304095
3,ec78d697-017f-11ed-9366-c1f5ac1e100e,1657588446835,1657588451420,SUCCESS,Content-Based Routing,94a5cf10-3bd5-11ec-815e-a6c4ac1e124d,AS2,243,AS2,3b39c580-3b91-11ea-bc40-05bbac1b495c,3901,4585,1657588450736
4,17c5e40b-0180-11ed-84c3-49e4ac1e100f,1657588519220,1657588522663,SUCCESS,MaKo,3bf5d2d0-14df-11ea-ba2f-ae43ac1b495c_df11a820-...,AS2,199,AS2,40d50320-f563-11e9-9e7f-fa2cac1b495c,3397,3443,1657588522617
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7493,2972430a-017c-11ed-84c3-49e4ac1e100f,1657586830258,1657586834018,SUCCESS,MAKO_BE_ROUTER,00fed140-e0f1-11e8-be62-528eac1b495c,AS2,1962,AS2,4c508960-e28e-11e8-be62-528eac1b495c,3714,3760,1657586833972
7494,7bb458f0-017d-11ed-bf3d-7778ac1e100d,1657587398263,1657587403841,SUCCESS,Metadata-Based Routing,04dcce90-d813-11eb-b670-1c1aac1e124d,SLMP_PLUS_MS,4418,AS2,0dd965c0-843f-11ec-9bc2-dedeac1e124d_352fda10-...,5537,5578,1657587403800
7495,f8643554-017d-11ed-9366-c1f5ac1e100e,1657587607946,1657587611091,SUCCESS,Metadata-Based Routing,0a5f23d0-355d-11eb-98c1-b429ac1b495c,SLMP_PLUS_MS,1288,AS2,bbaed6b1-437e-11eb-b000-d76cac1b495c_0ef2c320-...,3091,3145,1657587611037
7496,7fbb1eb3-017e-11ed-9366-c1f5ac1e100e,1657587839836,1657587848031,SUCCESS,Metadata-Based Routing,0dd965c0-843f-11ec-9bc2-dedeac1e124d_352fda10-...,AS2,4799127,SLMP_PLUS_MS,04dcce90-d813-11eb-b670-1c1aac1e124d,8164,8195,1657587848000


In [10]:
df_pandas['CSTARTTIME2']=(df_pandas['CSTARTTIME']/1000).astype(int)

df_pandas['CSTARTTIME']=pd.to_datetime(df_pandas['CSTARTTIME2'],unit='s')
df_pandas['CMESSAGETAT2']=df_pandas['CMESSAGETAT2']

df.printSchema()
df_pandas

root
 |-- CGLOBALMESSAGEID: string (nullable = true)
 |-- CSTARTTIME: decimal(38,0) (nullable = true)
 |-- CENDTIME: decimal(38,0) (nullable = true)
 |-- CSTATUS: string (nullable = true)
 |-- CSERVICE: string (nullable = true)
 |-- CSENDERENDPOINTID: string (nullable = true)
 |-- CSENDERPROTOCOL: string (nullable = true)
 |-- CINBOUNDSIZE: decimal(38,0) (nullable = true)
 |-- CRECEIVERPROTOCOL: string (nullable = true)
 |-- CRECEIVERENDPOINTID: string (nullable = true)
 |-- CSLATAT: decimal(38,0) (nullable = true)
 |-- CMESSAGETAT2: decimal(38,0) (nullable = true)
 |-- CSLADELIVERYTIME: decimal(38,0) (nullable = true)



,CGLOBALMESSAGEID,CSTARTTIME,CENDTIME,CSTATUS,CSERVICE,CSENDERENDPOINTID,CSENDERPROTOCOL,CINBOUNDSIZE,CRECEIVERPROTOCOL,CRECEIVERENDPOINTID,CSLATAT,CMESSAGETAT2,CSLADELIVERYTIME,CSTARTTIME2
0,d02228b0-017e-11ed-84c3-49e4ac1e100f,2022-07-12 01:06:09,1657587973838,SUCCESS,MAKO_BE_ROUTER,e57c94c0-e0f0-11e8-be62-528eac1b495c,AS2,198,AS2,9eeb0b20-a253-11e9-a189-ddafac1b495c,4302,4389,1657587973751,1657587969
1,4df52828-017f-11ed-84c3-49e4ac1e100f,2022-07-12 01:09:40,1657588186259,SUCCESS,MAKO_BE_ROUTER,e57c94c0-e0f0-11e8-be62-528eac1b495c,AS2,185,AS2,88d4b600-3019-11ec-8a09-23efac1e124c,5456,5487,1657588186228,1657588180
2,95cfeea3-017f-11ed-9366-c1f5ac1e100e,2022-07-12 01:11:41,1657588304140,SUCCESS,MAKO_BE_ROUTER,e57c94c0-e0f0-11e8-be62-528eac1b495c,AS2,65575,REST,f9641661-3874-11eb-98c1-b429ac1b495c,3006,3051,1657588304095,1657588301
3,ec78d697-017f-11ed-9366-c1f5ac1e100e,2022-07-12 01:14:06,1657588451420,SUCCESS,Content-Based Routing,94a5cf10-3bd5-11ec-815e-a6c4ac1e124d,AS2,243,AS2,3b39c580-3b91-11ea-bc40-05bbac1b495c,3901,4585,1657588450736,1657588446
4,17c5e40b-0180-11ed-84c3-49e4ac1e100f,2022-07-12 01:15:19,1657588522663,SUCCESS,MaKo,3bf5d2d0-14df-11ea-ba2f-ae43ac1b495c_df11a820-...,AS2,199,AS2,40d50320-f563-11e9-9e7f-fa2cac1b495c,3397,3443,1657588522617,1657588519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7493,2972430a-017c-11ed-84c3-49e4ac1e100f,2022-07-12 00:47:10,1657586834018,SUCCESS,MAKO_BE_ROUTER,00fed140-e0f1-11e8-be62-528eac1b495c,AS2,1962,AS2,4c508960-e28e-11e8-be62-528eac1b495c,3714,3760,1657586833972,1657586830
7494,7bb458f0-017d-11ed-bf3d-7778ac1e100d,2022-07-12 00:56:38,1657587403841,SUCCESS,Metadata-Based Routing,04dcce90-d813-11eb-b670-1c1aac1e124d,SLMP_PLUS_MS,4418,AS2,0dd965c0-843f-11ec-9bc2-dedeac1e124d_352fda10-...,5537,5578,1657587403800,1657587398
7495,f8643554-017d-11ed-9366-c1f5ac1e100e,2022-07-12 01:00:07,1657587611091,SUCCESS,Metadata-Based Routing,0a5f23d0-355d-11eb-98c1-b429ac1b495c,SLMP_PLUS_MS,1288,AS2,bbaed6b1-437e-11eb-b000-d76cac1b495c_0ef2c320-...,3091,3145,1657587611037,1657587607
7496,7fbb1eb3-017e-11ed-9366-c1f5ac1e100e,2022-07-12 01:03:59,1657587848031,SUCCESS,Metadata-Based Routing,0dd965c0-843f-11ec-9bc2-dedeac1e124d_352fda10-...,AS2,4799127,SLMP_PLUS_MS,04dcce90-d813-11eb-b670-1c1aac1e124d,8164,8195,1657587848000,1657587839


In [12]:
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import time


df_pandas.plot(kind='line',x='CSTARTTIME',y='CMESSAGETAT2')


TypeError: no numeric data to plot